# Ejemplos de SimPy

In [16]:
#!pip install simpy

In [17]:
import random
import simpy

## [Abandono bancario](https://simpy.readthedocs.io/en/latest/examples/bank_renege.html)
**Cubierta:**

* Recursos: _resource_
* Eventos de condición

**Escenario:**
>Un cajero con un tiempo de servicio aleatorio y clientes que abandonan.

Cada cliente tiene cierta paciencia. Espera llegar a la caja hasta que sea atendido. Si llega a la caja, está ahí por un tiempo y luego se va cuando sea atendido.
Los nuevos clientes son creados por la función _source_ cada cierto tiempo.

In [18]:
RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Total de clientes
INTERVAL_CUSTOMERS = 10.0  # Genera nuevos clientes aproximadamente cada x segundos
MIN_PATIENCE = 1  # Min. paciencia de cliente
MAX_PATIENCE = 3  # Max. paciencia de cliente


def source(env, number, interval, counter):
    """Fuente generadora de clientes aleatoriamente"""
    for i in range(number):
        c = customer(env, 'Cliente%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer(env, name, counter, time_in_bank):
    """El cliente llega, es atendido y sale."""
    arrive = env.now
    print('%7.4f %s: aquí estoy' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Espera hasta la caja y abandona la cola
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # A la caja
            print('%7.4f %s: esperé %6.3f' % (env.now, name, wait))

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: terminé' % (env.now, name))

        else:
            # Abandonar la cola
            print('%7.4f %s: me cansé después de %6.3f' % (env.now, name, wait))


# Prepara e inicia la simulación
print('Abandono bancario')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Empezar proceso y corre
counter = simpy.Resource(env, capacity=1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()

Abandono bancario
 0.0000 Cliente00: aquí estoy
 0.0000 Cliente00: esperé  0.000
 3.8595 Cliente00: terminé
10.2006 Cliente01: aquí estoy
10.2006 Cliente01: esperé  0.000
12.7265 Cliente02: aquí estoy
13.9003 Cliente02: me cansé después de  1.174
23.7507 Cliente01: terminé
34.9993 Cliente03: aquí estoy
34.9993 Cliente03: esperé  0.000
37.9599 Cliente03: terminé
40.4798 Cliente04: aquí estoy
40.4798 Cliente04: esperé  0.000
43.1401 Cliente04: terminé


## [Abandono de cine](https://simpy.readthedocs.io/en/latest/examples/movie_renege.html)
**Cubierta:**

* Recursos: _resource_
* Eventos de condición
* Eventos compartidos

**Escenario:**

>Una sala de cine tiene un mostrador de venta de entradas (caja) para tres películas (la próxima función). Cuando una película está agotada, todas las personas que esperan comprar entradas para esa película abandonan la cola.

Las personas llegan a veces al azar y compran un número aleatorio (1-6) de boletas para una película al azar. Cuando una película está agotada, todas las personas que esperan comprar un boleto para esa película abandonan la fila.

La sala de cine es solo un contenedor de todos los datos relacionados (películas, el cajero, boletas, datos recopilados, ...). El cajero es un _resource_ con una capacidad de uno.

La función _moviegoer_ comienza a esperar hasta que sea su turno (adquiere la caja que es un _resource_) o hasta que se active la señal de agotado. Si este último es el caso, abandona la cola. Si llega al mostrador, intenta comprar algunas boletas. Esto podría no ser exitoso, por ejemplo, si el proceso intenta comprar 5 boletos, solo quedan 3. Si quedan menos dos boletos después de la compra del boleto, se activa la señal de agotado.

Los espectadores (clientes) son generados por la función _customer arrivals_. También elige una película y el número de entradas para el espectador.


In [19]:
import collections

RANDOM_SEED = 42
TICKETS = 50  # Número de boletas por película
SIM_TIME = 120  # Tiempo de simulación


def moviegoer(env, movie, num_tickets, theater):
    """Un cliente intenta comprar una cantidad de boletas (*num_tickets*)
    para una determinada película (*movie*) en un teatro (*theater*).

   Si la película se agota, sale del teatro. Si llega a la caja intenta comprar
    algunas boletas. Si no hay suficientes boletas, el cliente discute y se va.

    Si al menos una boleta queda después de que el cliente compra las suyas
    el evento *sold out* se activa para esa película haciendo que
    todos los clientes que están en la cola se vayan.

    """
    with theater.counter.request() as my_turn:
        # Espera hasta que sea nuestro turno o la película se haya agotado
        result = yield my_turn | theater.sold_out[movie]

        # Comprueba si es nuestro turno si la película está agotada
        if my_turn not in result:
            theater.num_renegers[movie] += 1
            env.exit()

        # Comprueba si quedan suficientes boletas
        if theater.available[movie] < num_tickets:
            # Espectadores salen después de alguna discusión
            yield env.timeout(0.5)
            env.exit()

        # Comprar boletas
        theater.available[movie] -= num_tickets
        if theater.available[movie] < 2:
            # Activa el evento sold_out (agotado) para la película
            theater.sold_out[movie].succeed()
            theater.when_sold_out[movie] = env.now
            theater.available[movie] = 0
        yield env.timeout(1)


def customer_arrivals(env, theater):
    """Crea nuevos espectadores hasta que el simulador alcance el tiempo 120"""
    while True:
        yield env.timeout(random.expovariate(1 / 0.5))

        movie = random.choice(theater.movies)
        num_tickets = random.randint(1, 6)
        if theater.available[movie]:
            env.process(moviegoer(env, movie, num_tickets, theater))


Theater = collections.namedtuple('Theater', 'counter, movies, available, '
                                            'sold_out, when_sold_out, '
                                            'num_renegers')


# Preparar y empezar la simulación
print('Abandono de cine')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Crear cine
counter = simpy.Resource(env, capacity=1)
movies = ['Python Unchained', 'Kill Process', 'Pulp Implementation']
available = {movie: TICKETS for movie in movies}
sold_out = {movie: env.event() for movie in movies}
when_sold_out = {movie: None for movie in movies}
num_renegers = {movie: 0 for movie in movies}
theater = Theater(counter, movies, available, sold_out, when_sold_out,
                  num_renegers)

# Comenzar funciones y ejecutar
env.process(customer_arrivals(env, theater))
env.run(until=SIM_TIME)

# Análisis/resultados
for movie in movies:
    if theater.sold_out[movie]:
        print('Película "%s" agotada %.1f minutos después de abrir la caja' % (movie, theater.when_sold_out[movie]))
        print('  Número de personas que salen de la cola cuando se agota la película: %s' %
              theater.num_renegers[movie])

Abandono de cine
Película "Python Unchained" agotada 38.0 minutos después de abrir la caja
  Número de personas que salen de la cola cuando se agota la película: 16
Película "Kill Process" agotada 43.0 minutos después de abrir la caja
  Número de personas que salen de la cola cuando se agota la película: 5
Película "Pulp Implementation" agotada 28.0 minutos después de abrir la caja
  Número de personas que salen de la cola cuando se agota la película: 5


## [Tienda de máquinas](https://simpy.readthedocs.io/en/latest/examples/machine_shop.html)
**Cubierta:**

* Interrupción
* Recursos: _PreemptiveResource_

**Escenario:**
>Un taller tiene n máquinas iguales. Llegan varios trabajos para realizar con las máquinas y todas quedan ocupadas. Cada máquina se descompone periódicamente. Cada máquina se repara con un reparador. El reparador tiene tareas (aunque menos importantes) que realizar. Esas tareas se priorizan para las máquinas dañadas. El reparador continúa las tareas cuando termina de reparar la máquina. El taller funciona continuamente.

Una máquina tiene dos funciones:

_working_ implementa el comportamiento real de la máquina (producción de piezas). 

_break machine_ interrumpe periódicamente la función _working_ para simular la falla de la máquina.

El otro trabajo del reparador también es un proceso (implementado por other_job). El reparador en sí mismo es un _PreemptiveResource_ con una capacidad de 1. La reparación de la máquina tiene una prioridad de 1, mientras que el otro trabajo tiene una prioridad de 2 (cuanto menor es el número, mayor es la prioridad).

In [20]:
RANDOM_SEED = 42
PT_MEAN = 10.0         # Promedio de tiempo de procesamiento en minutos
PT_SIGMA = 2.0         # Sigma del tiempo de procesamiento
MTTF = 300.0           # Tiempo de fallo en minutos
BREAK_MEAN = 1 / MTTF  # Parámetro para distribución expovariate (random.expovariate)
REPAIR_TIME = 30.0     # Tiempo que toma para reparar la máquina en minutos
JOB_DURATION = 30.0    # Duración de otros trabajos en minutos
NUM_MACHINES = 10      # Número de máquinas en la tienda
WEEKS = 4              # Tiempo de simulación en semanas
SIM_TIME = WEEKS * 7 * 24 * 60  # Tiempo de simulación en minutos


def time_per_part():
    """Retorna el tiempo de procesamiento actual para una pieza de concreto."""
    return random.normalvariate(PT_MEAN, PT_SIGMA)


def time_to_failure():
    """Devuelve el tiempo hasta el próximo fallo para una máquina."""
    return random.expovariate(BREAK_MEAN)


class Machine(object):
    """Una máquina produce partes y se rompen de vez en cuando.

    Si se rompe, pide un reparador (*repairman*) y continúa la producción cuando la reparen

    Una máquina tiene un nombre (*name*) y un número de partes *parts_made* que lleva hechas.

    """
    def __init__(self, env, name, repairman):
        self.env = env
        self.name = name
        self.parts_made = 0
        self.broken = False

        # Iniciar funciones "working" y "break_machine" para esta máquina.
        self.process = env.process(self.working(repairman))
        env.process(self.break_machine())

    def working(self, repairman):
        """Produce partes a lo largo de la ejecución de la simulación.

        Mientras hace una parte, la máquina puede romper muchas veces.
        Pide al reparador cuando pase esto.

        """
        while True:
            # Empieza haciendo una nueva parte
            done_in = time_per_part()
            while done_in:
                try:
                    # Trabaja para la parte
                    start = self.env.now
                    yield self.env.timeout(done_in)
                    done_in = 0  # Asigna 0 para salir del ciclo

                except simpy.Interrupt:
                    self.broken = True
                    done_in -= self.env.now - start  # cuánto tiempo queda?

                    # Pedir un reparador. Esto interrumpe other_job por este trabajo
                    with repairman.request(priority=1) as req:
                        yield req
                        yield self.env.timeout(REPAIR_TIME)

                    self.broken = False

            # La parte está hecha
            self.parts_made += 1

    def break_machine(self):
        """Rompe la máquina de vez en cuando."""
        while True:
            yield self.env.timeout(time_to_failure())
            if not self.broken:
                # Romper la máquina si esta está funcionando actualmente.
                self.process.interrupt()


def other_jobs(env, repairman):
    """El otro trabajo (menos importante) del reparador"""
    while True:
        # Comenzar el nuevo trabajo
        done_in = JOB_DURATION
        while done_in:
            # Reintentar el trabajo hasta que esté hecho.
            # Su prioridad es menor que la reparación de la máquina
            with repairman.request(priority=2) as req:
                yield req
                try:
                    start = env.now
                    yield env.timeout(done_in)
                    done_in = 0
                except simpy.Interrupt:
                    done_in -= env.now - start


# Prepara e inicia la simulación
print('Tienda de máquinas')
random.seed(RANDOM_SEED)  # Ayuda a producir los mismos resultados

# Crea un environment y empieza a preparar
env = simpy.Environment()
repairman = simpy.PreemptiveResource(env, capacity=1)
machines = [Machine(env, 'Máquina %d' % i, repairman)
            for i in range(NUM_MACHINES)]
env.process(other_jobs(env, repairman))

# Ejecuta!
env.run(until=SIM_TIME)

# Análisis/resultados
print('El resultado de la tienda de máquinas después de %s semanas' % WEEKS)
for machine in machines:
    print('%s hechas %d partes.' % (machine.name, machine.parts_made))

Tienda de máquinas
El resultado de la tienda de máquinas después de 4 semanas
Máquina 0 hechas 3251 partes.
Máquina 1 hechas 3273 partes.
Máquina 2 hechas 3242 partes.
Máquina 3 hechas 3343 partes.
Máquina 4 hechas 3387 partes.
Máquina 5 hechas 3244 partes.
Máquina 6 hechas 3269 partes.
Máquina 7 hechas 3185 partes.
Máquina 8 hechas 3302 partes.
Máquina 9 hechas 3279 partes.


## [Reabastecimiento de gasolinera](https://simpy.readthedocs.io/en/latest/examples/gas_station_refuel.html)
**Cubierta:**

* Recursos: _Resource_, _Container_
* Esperar otros procesos

**Escenario:**
>Una estación de servicio tiene un número limitado de bombas de gas que comparten un depósito de combustible común. Los autos llegan al azar a la estación de servicio, solicitan una de las bombas de combustible y comienzan a repostar combustible desde ese depósito.

>La función gas_station_control observa el nivel de combustible de la estación de servicio y llama a un camión cisterna para reponer combustible si el nivel de la estación cae por debajo de un umbral.

Este ejemplo modela una estación de servicio y autos que llegan a la estación para depositar combustible.

La estación de servicio tiene un número limitado de bombas de combustible y un tanque de combustible compartido entre las bombas de combustible. La gasolinera se modela como _Resource_. El tanque de combustible compartido está modelado con un _Container_.

Los vehículos que llegan a la estación de servicio solicitan primero una bomba de combustible de la estación. Una vez que adquieren una, intentan tomar la cantidad deseada de combustible de la bomba de combustible. Se van cuando terminan.

El nivel de combustible de las estaciones de gas es controlado regularmente por _gas station control_. Cuando el nivel cae por debajo de un cierto umbral, se llama a un camión cisterna para reabastecer de combustible a la estación de servicio.

Cuando el nivel cae por debajo de un cierto umbral, se llama a un camión cisterna para reabastecer de combustible a la estación de servicio.

In [21]:
import itertools

RANDOM_SEED = 42
GAS_STATION_SIZE = 200     # Litros
THRESHOLD = 10             # Umbral para llamar al camión de llenado (en %)
FUEL_TANK_SIZE = 50        # Litros
FUEL_TANK_LEVEL = [5, 25]  # Min/max niveles de tanques de combustible (en litros)
REFUELING_SPEED = 2        # Litros/segundo
TANK_TRUCK_TIME = 300      # Secundos que toma el camión en llegar
T_INTER = [30, 300]        # Crea un carro cada [min, max] segundos
SIM_TIME = 1000            # Tiempo de simulación en segundos


def car(name, env, gas_station, fuel_pump):
    """Un automóvil llega a la estación de servicio para reabastecerse de combustible

    Pide a una de las bombas de combustible de la estación de servicio e intenta obtener la
    cantidad deseada de combustible de esta. Si el depósito de la estación está
    agotado, el carro tiene que esperar a que el carrotanque llegue.

    """
    fuel_tank_level = random.randint(*FUEL_TANK_LEVEL)
    print('%s llega a la estación en %.1f' % (name, env.now))
    with gas_station.request() as req:
        start = env.now
        # Pide una de las bombas de combustible
        yield req

        # Obtiene el monto requerido de combustible
        liters_required = FUEL_TANK_SIZE - fuel_tank_level
        yield fuel_pump.get(liters_required)

        # El proceso de reabastecimiento "actual" se demora un tiempo
        yield env.timeout(liters_required / REFUELING_SPEED)

        print('%s terminada la recarga en %.1f segundos.' % (name,
                                                          env.now - start))


def gas_station_control(env, fuel_pump):
    """Periódicamente revisa el nivel de la bomba de combustible (*fuel_pump*) y llama al
    carrotanque si el nivel cae por debajo del límite o umbral"""
    while True:
        if fuel_pump.level / fuel_pump.capacity * 100 < THRESHOLD:
            # Necesitamos llamar al carrotanque ahora!
            print('Llamar el tanque en %d' % env.now)
            # Espera que el carrotanque llegue y reabastezca la estación
            yield env.process(tank_truck(env, fuel_pump))

        yield env.timeout(10)  # Revisar cada 10 segundos


def tank_truck(env, fuel_pump):
    """Llega a la gasolinera después de cierto retraso y la reabastece."""
    yield env.timeout(TANK_TRUCK_TIME)
    print('El carrotanque llega en el tiempo %d' % env.now)
    ammount = fuel_pump.capacity - fuel_pump.level
    print('El carrotanque reabastece %.1f litros.' % ammount)
    yield fuel_pump.put(ammount)


def car_generator(env, gas_station, fuel_pump):
    """Genera carros nuevos que llegan a la gasolinera."""
    for i in itertools.count():
        yield env.timeout(random.randint(*T_INTER))
        env.process(car('Carro %d' % i, env, gas_station, fuel_pump))


# Prepara e inicia la simulación
print('Reabastecimiento de gasolinera')
random.seed(RANDOM_SEED)

# Crea un environment e inicia los procesos
env = simpy.Environment()
gas_station = simpy.Resource(env, 2)
fuel_pump = simpy.Container(env, GAS_STATION_SIZE, init=GAS_STATION_SIZE)
env.process(gas_station_control(env, fuel_pump))
env.process(car_generator(env, gas_station, fuel_pump))

# Ejecutar!
env.run(until=SIM_TIME)

Reabastecimiento de gasolinera
Carro 0 llega a la estación en 87.0
Carro 0 terminada la recarga en 18.5 segundos.
Carro 1 llega a la estación en 129.0
Carro 1 terminada la recarga en 19.0 segundos.
Carro 2 llega a la estación en 284.0
Carro 2 terminada la recarga en 21.0 segundos.
Carro 3 llega a la estación en 385.0
Carro 3 terminada la recarga en 13.5 segundos.
Carro 4 llega a la estación en 459.0
Llamar el tanque en 460
Carro 4 terminada la recarga en 22.0 segundos.
Carro 5 llega a la estación en 705.0
Carro 6 llega a la estación en 750.0
El carrotanque llega en el tiempo 760
El carrotanque reabastece 188.0 litros.
Carro 6 terminada la recarga en 29.0 segundos.
Carro 5 terminada la recarga en 76.5 segundos.
Carro 7 llega a la estación en 891.0
Carro 7 terminada la recarga en 13.0 segundos.


## [Latencia del evento](https://simpy.readthedocs.io/en/latest/examples/latency.html)
**Cubierta:**

* Recursos: _Store_

**Escenario:**
>Este ejemplo muestra cómo separar el tiempo de espera de eventos entre procesos y los procesos mismos.

Al modelar cosas físicas como cables, propagación RF, etc., es mejor encapsular para mantener este mecanismo de propagación fuera de los procesos de envío y recepción.

También se puede interconectar procesos enviando mensajes.

In [22]:
SIM_DURATION = 100


class Cable(object):
    """Esta clase representa la propagación que atraviesa un cable."""
    def __init__(self, env, delay):
        self.env = env
        self.delay = delay
        self.store = simpy.Store(env)

    def latency(self, value):
        yield self.env.timeout(self.delay)
        self.store.put(value)

    def put(self, value):
        self.env.process(self.latency(value))

    def get(self):
        return self.store.get()


def sender(env, cable):
    """Un proceso que genera mensajes aleatorios."""
    while True:
        # Espera la siguiente transmisión
        yield env.timeout(5)
        cable.put('remitente envió esto en %d' % env.now)


def receiver(env, cable):
    """Un proceso que gasta mensajes."""
    while True:
        # Obtener evento para canalizar mensajes
        msg = yield cable.get()
        print('Recibido esto en %d mientras %s' % (env.now, msg))


# Preparar y empezar simulación
print('Latencia de eventos')
env = simpy.Environment()

cable = Cable(env, 10)
env.process(sender(env, cable))
env.process(receiver(env, cable))

env.run(until=SIM_DURATION)

Latencia de eventos
Recibido esto en 15 mientras remitente envió esto en 5
Recibido esto en 20 mientras remitente envió esto en 10
Recibido esto en 25 mientras remitente envió esto en 15
Recibido esto en 30 mientras remitente envió esto en 20
Recibido esto en 35 mientras remitente envió esto en 25
Recibido esto en 40 mientras remitente envió esto en 30
Recibido esto en 45 mientras remitente envió esto en 35
Recibido esto en 50 mientras remitente envió esto en 40
Recibido esto en 55 mientras remitente envió esto en 45
Recibido esto en 60 mientras remitente envió esto en 50
Recibido esto en 65 mientras remitente envió esto en 55
Recibido esto en 70 mientras remitente envió esto en 60
Recibido esto en 75 mientras remitente envió esto en 65
Recibido esto en 80 mientras remitente envió esto en 70
Recibido esto en 85 mientras remitente envió esto en 75
Recibido esto en 90 mientras remitente envió esto en 80
Recibido esto en 95 mientras remitente envió esto en 85


## [Comunicación de procesos](https://simpy.readthedocs.io/en/latest/examples/process_communication.html)
**Cubierta:**

* Recursos: _Store_

**Escenario:**
>Este ejemplo muestra cómo interconectar elementos de modelos de simulación usando "resources.Store" para procesos asíncronos uno a uno y muchos a uno. Para uno a muchos, una simple clase BroadCastPipe es construída de Store.

Cuando un consumidor de procesos no siempre espera en un proceso de generación y estos procesos corren asincrónicamente. Este ejemplo muestra cómo crear un búfer y también decir si el proceso del consumidor se retrasó en el evento debido a un proceso de generación.

Esto también es útil cuando es necesario transmitir cierta información a muchos procesos de recepción.
Finalmente, el uso de tuberías puede simplificar la forma en que los procesos se interconectan entre sí en un modelo de simulación.

In [23]:
RANDOM_SEED = 42
SIM_TIME = 100


class BroadcastPipe(object):
    """Una tubería Broadcast que dispone de un proceso para enviar mensajes a muchos.

    Este constructor es usado cuando los consumidores de mensajes están ejecutando a
    diferentes velocidades que los generadores de mensajes y proporcionan un búfer
    de eventos a los consumidores de procesos.

    Los parámetros son usados para crear una nueva
    :class:`~simpy.resources.store.Store` instancia cada cierto tiempo
    :meth:`get_output_conn()` es llamada.

    """
    def __init__(self, env, capacity=simpy.core.Infinity):
        self.env = env
        self.capacity = capacity
        self.pipes = []

    def put(self, value):
        """Difundir un valor (*value*) a todos los receptores."""
        if not self.pipes:
            raise RuntimeError('No hay tubos de salida.')
        events = [store.put(value) for store in self.pipes]
        return self.env.all_of(events)  # Condición de eventos para todos los "events"

    def get_output_conn(self):
        """Obtiene una nueva conexión de salida para este tubo de difusión.

        El valor de retorno es una :class:`~simpy.resources.store.Store`.

        """
        pipe = simpy.Store(self.env, capacity=self.capacity)
        self.pipes.append(pipe)
        return pipe


def message_generator(name, env, out_pipe):
    """Un proceso que genera mensajes aleatorios."""
    while True:
        # Espera la próxima transmisión
        yield env.timeout(random.randint(6, 10))

        # Los mensajes son marcados con el tiempo para verificar más tarde si el consumidor llegó
        # tarde para obtenerlos.  Nota, usando event.triggered puede hacer que
        # resulte un fallo debido a la naturaleza de FIFO de los rendimientos de simulación
        # Es decir, si el mismo env.now, message_generator pone un mensaje
        # en la primera tubería y luego message_consumer llega desde la tubería,
        # el event.triggered será True, en otro caso será False
        msg = (env.now, '%s decir Hola a %d' % (name, env.now))
        out_pipe.put(msg)


def message_consumer(name, env, in_pipe):
    """Un proceso que consume mensajes."""
    while True:
        # Obtener evento para tubería de mensajes
        msg = yield in_pipe.get()

        if msg[0] < env.now:
            # Si el mensaje ya fue puesto dentro de la tubería, entonces
            # message_consumer llega tarde a él. Dependiendo en que
            # está siendo modelado esto, puede, o no puede tener algún significado
            print('Mensaje retrasado: en el tiempo %d: %s recibió mensaje: %s' %
                  (env.now, name, msg[1]))

        else:
            # message_consumer está sincronizado con message_generator
            print('en el tiempo %d: %s recibió mensaje: %s.' %
                  (env.now, name, msg[1]))

        # El proceso hace algún otro trabajo, que puede resultar en mensajes de espera
        yield env.timeout(random.randint(4, 8))


# Preparar e iniciar simulación
print('Comunicación de procesos')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Para tuberías tipo 1 a 1 y 1 a muchos, usar Store
pipe = simpy.Store(env)
env.process(message_generator('Generador A', env, pipe))
env.process(message_consumer('Consumidor A', env, pipe))

print('\nComunicación de tuberías 1 a 1\n')
env.run(until=SIM_TIME)

# para 1 a muchos use BroadcastPipe
# (Note: También debe ser usado para 1 a 1, muchos a uno o muchos a muchos)
env = simpy.Environment()
bc_pipe = BroadcastPipe(env)

env.process(message_generator('Generador A', env, bc_pipe))
env.process(message_consumer('Consumidor A', env, bc_pipe.get_output_conn()))
env.process(message_consumer('Consumidor B', env, bc_pipe.get_output_conn()))

print('\nComunicación de tuberías 1 a muchos\n')
env.run(until=SIM_TIME)

Comunicación de procesos

Comunicación de tuberías 1 a 1

en el tiempo 6: Consumidor A recibió mensaje: Generador A decir Hola a 6.
en el tiempo 12: Consumidor A recibió mensaje: Generador A decir Hola a 12.
en el tiempo 19: Consumidor A recibió mensaje: Generador A decir Hola a 19.
en el tiempo 26: Consumidor A recibió mensaje: Generador A decir Hola a 26.
en el tiempo 36: Consumidor A recibió mensaje: Generador A decir Hola a 36.
en el tiempo 46: Consumidor A recibió mensaje: Generador A decir Hola a 46.
en el tiempo 52: Consumidor A recibió mensaje: Generador A decir Hola a 52.
en el tiempo 58: Consumidor A recibió mensaje: Generador A decir Hola a 58.
Mensaje retrasado: en el tiempo 66: Consumidor A recibió mensaje: Generador A decir Hola a 65
en el tiempo 75: Consumidor A recibió mensaje: Generador A decir Hola a 75.
en el tiempo 85: Consumidor A recibió mensaje: Generador A decir Hola a 85.
en el tiempo 95: Consumidor A recibió mensaje: Generador A decir Hola a 95.

Comunicación 

## [Lavadero de autos](https://simpy.readthedocs.io/en/latest/examples/carwash.html)
**Cubierta:**

* Recursos: _Resource_
* Esperando de otros procesos

**Escenario:**
>Un lavadero de autos tiene una cantidad limitada de máquinas de lavado y definen una función de lavado que toma un tiempo aleatorio. La función car llega al lavadero en un tiempo aleatorio. Si una máquina de lavado está disponible, empiezan la función de lavado y esperan hasta finalizar. Si no, esperan hasta que se desocupe uno.

El ejemplo lavadero de autos es una simulación de un lavadero con un número limitado de máquinas y un número de carros que llegan al lavadero para ser lavados.

El lavadero usa un _Resource_ para modelar el número limitado de máquinas de lavado. También define una función para lavar el auto.

Cuando el auto llega al lavadero, pide una máquina. Una vez que la obtenga, inicia la función wash de carwash y espera hasta que termine. Finalmente termina la máquina y sale.

Los autos son generados por la función setup. Después crea una cantidad inicial de autos y la función car crea esos nuevos autos después de un intervalo de tiempo aleatorio a lo largo de la simulación.

In [24]:
RANDOM_SEED = 42
NUM_MACHINES = 2  # Número de máquinas en el lavadero
WASHTIME = 5      # Minutos que toma en lavar un auto
T_INTER = 7       # Crea un carro cada aproximadamente 7 minutos
SIM_TIME = 20     # Tiempo de simulación en minutos


class Carwash(object):
    """Un lavadero de autos tiene un número limitado de máquinas (``NUM_MACHINES``) para
    lavar carros en paralelo.

    Los carros tienen que pedir una de las máquinas. Cuando la obtengan
    puede iniciar la función washing y esperan hasta que termine
    (que toma ``washtime`` minutos).

    """
    def __init__(self, env, num_machines, washtime):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.washtime = washtime

    def wash(self, car):
        """Proceso de lavado. Toma la función car y empieza a lavarlo."""
        yield self.env.timeout(WASHTIME)
        print("Lavadero eliminó %d%% de suciedad del %s" %
              (random.randint(50, 99), car))


def car(env, name, cw):
    """La función car (cada carro tiene un nombre ``name``) llega al lavadero
    (``cw``) y pide una máquina lavadora.

    La máquina inicia la función de lavado, espera a que termine y
    sale y no vuelve nunca más...

    """
    print('%s llega al lavadero en %.2f.' % (name, env.now))
    with cw.machine.request() as request:
        yield request

        print('%s entra al lavadero en %.2f.' % (name, env.now))
        yield env.process(cw.wash(name))

        print('%s sale del lavadero en %.2f.' % (name, env.now))


def setup(env, num_machines, washtime, t_inter):
    """Crea un lavadero, un número inicial de carros y todo el tiempo crea carros
    aproximadamente cada ``t_inter`` minutos."""
    # Crea el lavadero
    carwash = Carwash(env, num_machines, washtime)

    # Crea 4 carros iniciales
    for i in range(4):
        env.process(car(env, 'Carro %d' % i, carwash))

    # Crea más carros mientras corre la simulación
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        i += 1
        env.process(car(env, 'Carro %d' % i, carwash))


# Prepara e inicia la simulación
print('Lavadero de autos')
print('Revisa http://youtu.be/fXXmeP9TvBg mientras se simula... ;-)')
random.seed(RANDOM_SEED)  # Ayuda a reproducir los mismos resultados

# Crea un environment e inicia la función setup
env = simpy.Environment()
env.process(setup(env, NUM_MACHINES, WASHTIME, T_INTER))

# Ejecuta!
env.run(until=SIM_TIME)

Lavadero de autos
Revisa http://youtu.be/fXXmeP9TvBg mientras se simula... ;-)
Carro 0 llega al lavadero en 0.00.
Carro 1 llega al lavadero en 0.00.
Carro 2 llega al lavadero en 0.00.
Carro 3 llega al lavadero en 0.00.
Carro 0 entra al lavadero en 0.00.
Carro 1 entra al lavadero en 0.00.
Carro 4 llega al lavadero en 5.00.
Lavadero eliminó 97% de suciedad del Carro 0
Lavadero eliminó 67% de suciedad del Carro 1
Carro 0 sale del lavadero en 5.00.
Carro 1 sale del lavadero en 5.00.
Carro 2 entra al lavadero en 5.00.
Carro 3 entra al lavadero en 5.00.
Carro 5 llega al lavadero en 10.00.
Lavadero eliminó 64% de suciedad del Carro 2
Lavadero eliminó 58% de suciedad del Carro 3
Carro 2 sale del lavadero en 10.00.
Carro 3 sale del lavadero en 10.00.
Carro 4 entra al lavadero en 10.00.
Carro 5 entra al lavadero en 10.00.
Lavadero eliminó 97% de suciedad del Carro 4
Lavadero eliminó 56% de suciedad del Carro 5
Carro 4 sale del lavadero en 15.00.
Carro 5 sale del lavadero en 15.00.
Carro 6 llega 